<a href="https://colab.research.google.com/github/Gratisfo/One-media-sentiment/blob/main/sentiment_dostoevsky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install psycopg2-binary

In [ ]:
import psycopg2
from config import config
import pandas as pd


list_of_posts = []
list_of_ids = []
df_posts = pd.DataFrame()
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        # create a cursor
        cur = conn.cursor()
        cur.execute("""SELECT  message_id, content FROM message limit 500""")
        for message in cur.fetchall():
          list_of_ids.append(message[0])
          list_of_posts.append(message[1])
        df_posts['post'] = list_of_posts
        df_posts['id'] = list_of_ids
        cur.close()
        return df_posts, 
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


if __name__ == '__main__':
    connect()


Connecting to the PostgreSQL database...
Database connection closed.


In [ ]:
df_posts

,post,id
0,**Путин дал приказ после расстрела в пермском ...,1415220
1,**В авариях на дорогах ДНР травмированы три че...,1415219
2,@mk_rus,1415218
3,**В Кыргызстане прошли выборы депутатов в Госу...,1415217
4,"**Вынесен приговор жителю Кохмы, повторно упра...",1415216
...,...,...
495,**Огромный кит проплыл вплотную под SUP-бордом...,1414737
496,**Женщину-врача обвинили в убийстве троих доче...,1414736
497,"**Укропы, привет!** ВСН (ЛНР). #Донбасс",12761
498,**Башкирия запросила дополнительное финансиров...,1644


### Оценка `dostoevsky`

In [ ]:
!pip install dostoevsky

In [ ]:
!python -m dostoevsky download fasttext-social-network-model

In [ ]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
import fasttext
import pandas as pd

In [ ]:
fasttext.FastText.eprint = lambda x: None
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)
sentiment_list = []
results = model.predict(list_of_posts, k=2)
for sentiment in results:
    sentiment_list.append(sentiment)

In [ ]:
neutral_list = []
negative_list = []
positive_list = []
speech_list = []
skip_list = []
for sentiment in sentiment_list:
    neutral = sentiment.get('neutral')
    negative = sentiment.get('negative')
    positive = sentiment.get('positive')
    if neutral is None:
        neutral_list.append(0)
    else:
        neutral_list.append(sentiment.get('neutral'))
    if negative is None:
        negative_list.append(0)
    else:
        negative_list.append(sentiment.get('negative'))
    if positive is None:
        positive_list.append(0)
    else:
        positive_list.append(sentiment.get('positive'))
df_posts['neutral'] = neutral_list
df_posts['negative'] = negative_list
df_posts['positive'] = positive_list
df_posts

,post,id,neutral,negative,positive
0,**Путин дал приказ после расстрела в пермском ...,1415220,0.658428,0.206904,0.000000
1,**В авариях на дорогах ДНР травмированы три че...,1415219,0.629785,0.191943,0.000000
2,@mk_rus,1415218,1.000010,0.000000,0.001217
3,**В Кыргызстане прошли выборы депутатов в Госу...,1415217,0.749097,0.262852,0.000000
4,"**Вынесен приговор жителю Кохмы, повторно упра...",1415216,0.896261,0.160276,0.000000
...,...,...,...,...,...
495,**Огромный кит проплыл вплотную под SUP-бордом...,1414737,0.492198,0.168867,0.000000
496,**Женщину-врача обвинили в убийстве троих доче...,1414736,0.777310,0.206904,0.000000
497,"**Укропы, привет!** ВСН (ЛНР). #Донбасс",12761,0.000000,0.348655,0.000000
498,**Башкирия запросила дополнительное финансиров...,1644,0.787941,0.206904,0.000000


In [ ]:
df_posts[df_posts.negative > 0.5]

,post,id,neutral,negative,positive
329,"⚡️Оружие при нем было огнестрельным, а не трав...",16676,0.51563,0.52343,0.0
